In [ ]:
import math
from pathlib import Path
from typing import Tuple, Dict, Any

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

CSV_PATH = Path("runs.csv")
OUT_DIR  = Path("metricas")
OUT_DIR.mkdir(parents=True, exist_ok=True)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)
df = pd.read_csv(CSV_PATH)
df.head(1)
print("Linhas x colunas:", df.shape)


In [ ]:
def normalize_state(s):
    if pd.isna(s): return np.nan
    su = str(s).upper()
    if "DONE" in su: return "DONE"
    if "CANCEL" in su: return "CANCELLED"
    if "FAIL" in su or "ERROR" in su: return "FAILED"
    if "RUNNING" in su: return "RUNNING"
    return str(s)

df["state_norm"] = df.get("state", np.nan).apply(normalize_state)
df["success"]    = df["state_norm"].eq("DONE")

for col in ["timestamp_utc","start_ts","end_ts"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

num_cols = [
    "duration_s","throughput_rps","records_processed",
    "vcpu_hours","memory_gb_hours","pd_gb_hours","ssd_gb_hours",
    "shuffle_gb","total_shuffle_gb","current_vcpus","current_memory_gb",
    "cost_vcpu_usd","cost_memory_usd","cost_shuffle_usd",
    "total_cost_usd","unit_cost_usd_per_mm",
    "num_workers","max_num_workers"
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

def infer_test(row):
    for k in ["test","experiment"]:
        if k in row and pd.notna(row[k]):
            return str(row[k]).upper()
    import re
    m = re.search(r"(t\d+)", str(row.get("job_name","")), re.IGNORECASE)
    return m.group(1).upper() if m else np.nan

if "test" not in df.columns:
    df["test"] = df.apply(infer_test, axis=1)
else:
    df["test"] = df["test"].astype(str).str.upper()

if "variant" in df.columns:
    df["variant"] = df["variant"].astype(str).str.lower()

expected_min = ["test","variant","duration_s","throughput_rps","records_processed",
                "total_cost_usd","unit_cost_usd_per_mm","state_norm","success"]
audit = pd.DataFrame([(c, c in df.columns, (str(df[c].dtype) if c in df.columns else "-"))
                      for c in expected_min], columns=["column","present","dtype"])
audit_path = OUT_DIR / "audit_core_columns.csv"
audit.to_csv(audit_path, index=False)
audit


In [ ]:
def ci_mean_t(a: np.ndarray, alpha: float = 0.05) -> Tuple[float,float,float,int]:
    """Média e IC 95% via t-Student (robusto para n pequeno)."""
    a = a[~np.isnan(a)]
    n  = a.size
    m  = float(np.mean(a)) if n > 0 else np.nan
    sd = float(np.std(a, ddof=1)) if n > 1 else np.nan
    if n <= 1 or not math.isfinite(sd):
        return m, np.nan, np.nan, n
    tcrit = stats.t.ppf(1 - alpha/2, df=n-1)
    half  = tcrit * sd / math.sqrt(n)
    return m, m - half, m + half, n

def try_normality(a: np.ndarray) -> float:
    """Shapiro–Wilk, retorna p-valor (n>=5); NaN caso contrário."""
    a = a[~np.isnan(a)]
    if a.size < 5:
        return np.nan
    try:
        return stats.shapiro(a).pvalue
    except Exception:
        return np.nan

def test_two_groups(x: np.ndarray, y: np.ndarray) -> Dict[str, Any]:
    """Welch t-test se normalidade plausível e n>=5; senão Mann–Whitney.
       Retorna teste usado, estatística, p-valor e tamanho de efeito."""
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]
    nx, ny = x.size, y.size
    res = {"nx": int(nx), "ny": int(ny)}

    if nx < 2 or ny < 2:
        res.update({"test":"insuficiente", "stat":np.nan, "pvalue":np.nan,
                    "effect":np.nan, "effect_name":"-"})
        return res

    px = try_normality(x)
    py = try_normality(y)
    normal_enough = (not np.isnan(px) and px > 0.05) and (not np.isnan(py) and py > 0.05)

    if normal_enough and nx >= 5 and ny >= 5:
        # Welch
        t, p = stats.ttest_ind(x, y, equal_var=False)
        # Cohen's d
        sdx = np.var(x, ddof=1); sdy = np.var(y, ddof=1)
        sp  = np.sqrt(((nx-1)*sdx + (ny-1)*sdy) / (nx+ny-2)) if (nx+ny-2)>0 else np.nan
        d   = (np.mean(x) - np.mean(y)) / sp if (sp and sp>0) else np.nan
        res.update({"test":"Welch t", "stat":float(t), "pvalue":float(p),
                    "effect":float(d), "effect_name":"Cohen d"})
    else:
        # Mann–Whitney
        u, p = stats.mannwhitneyu(x, y, alternative="two-sided")
        r_rb = 1 - 2*u/(nx*ny)  # rank-biserial
        res.update({"test":"Mann-Whitney", "stat":float(u), "pvalue":float(p),
                    "effect":float(r_rb), "effect_name":"rank-biserial"})
    return res


In [ ]:
group_cols = ["test","variant"]
agg = {
    "success": ["count","sum"],
    "duration_s": ["mean","std","min","max","median"],
    "throughput_rps": ["mean","std","min","max","median"],
    "records_processed": ["mean","sum"],
    "total_cost_usd": ["mean","std","min","max","median","sum"],
    "unit_cost_usd_per_mm": ["mean","std","min","max","median"],
    "vcpu_hours": ["mean","sum"],
    "memory_gb_hours": ["mean","sum"],
    "shuffle_gb": ["mean","sum"]
}
present_agg = {k:v for k,v in agg.items() if k in df.columns}

summary = df.groupby(group_cols, dropna=False).agg(present_agg)
summary.columns = ["_".join([c for c in col if c]).strip("_") for col in summary.columns.values]
summary = summary.reset_index().rename(columns={
    "success_count":"n_runs", "success_sum":"n_sucessos",
    "duration_s_mean":"dur_mean_s","duration_s_std":"dur_std_s",
    "duration_s_min":"dur_min_s","duration_s_max":"dur_max_s","duration_s_median":"dur_mediana_s",
    "throughput_rps_mean":"thr_mean_rps","throughput_rps_std":"thr_std_rps",
    "throughput_rps_min":"thr_min_rps","throughput_rps_max":"thr_max_rps","throughput_rps_median":"thr_mediana_rps",
    "records_processed_mean":"reg_mean","records_processed_sum":"reg_total",
    "total_cost_usd_mean":"custo_medio_usd","total_cost_usd_std":"custo_std_usd",
    "total_cost_usd_min":"custo_min_usd","total_cost_usd_max":"custo_max_usd","total_cost_usd_median":"custo_mediana_usd",
    "total_cost_usd_sum":"custo_total_usd",
    "unit_cost_usd_per_mm_mean":"custo_unit_mean_usd_por_mm","unit_cost_usd_per_mm_std":"custo_unit_std_usd_por_mm",
    "unit_cost_usd_per_mm_min":"custo_unit_min_usd_por_mm","unit_cost_usd_per_mm_max":"custo_unit_max_usd_por_mm",
    "unit_cost_usd_per_mm_median":"custo_unit_mediana_usd_por_mm",
    "vcpu_hours_mean":"vcpu_mean_h","vcpu_hours_sum":"vcpu_total_h",
    "memory_gb_hours_mean":"mem_mean_gb_h","memory_gb_hours_sum":"mem_total_gb_h",
    "shuffle_gb_mean":"shuffle_mean_gb","shuffle_gb_sum":"shuffle_total_gb"
})
if "n_runs" in summary.columns and "n_sucessos" in summary.columns:
    summary["taxa_sucesso"] = summary["n_sucessos"] / summary["n_runs"]

summary_path = OUT_DIR / "resumo_teste_variante.csv"
summary.to_csv(summary_path, index=False)
summary


In [ ]:
# Nível Dataset (opcional): Teste–Variante–Dataset
if "dataset_tag" in df.columns:
    gds = ["test","variant","dataset_tag"]
    summary_ds = df.groupby(gds, dropna=False).agg(present_agg)
    summary_ds.columns = ["_".join([c for c in col if c]).strip("_") for col in summary_ds.columns.values]
    summary_ds = summary_ds.reset_index().rename(columns={
        "success_count":"n_runs", "success_sum":"n_sucessos",
        "duration_s_mean":"dur_mean_s","duration_s_std":"dur_std_s",
        "duration_s_min":"dur_min_s","duration_s_max":"dur_max_s","duration_s_median":"dur_mediana_s",
        "throughput_rps_mean":"thr_mean_rps","throughput_rps_std":"thr_std_rps",
        "throughput_rps_min":"thr_min_rps","throughput_rps_max":"thr_max_rps","throughput_rps_median":"thr_mediana_rps",
        "records_processed_mean":"reg_mean","records_processed_sum":"reg_total",
        "total_cost_usd_mean":"custo_medio_usd","total_cost_usd_std":"custo_std_usd",
        "total_cost_usd_min":"custo_min_usd","total_cost_usd_max":"custo_max_usd","total_cost_usd_median":"custo_mediana_usd",
        "total_cost_usd_sum":"custo_total_usd",
        "unit_cost_usd_per_mm_mean":"custo_unit_mean_usd_por_mm","unit_cost_usd_per_mm_std":"custo_unit_std_usd_por_mm",
        "unit_cost_usd_per_mm_min":"custo_unit_min_usd_por_mm","unit_cost_usd_per_mm_max":"custo_unit_max_usd_por_mm",
        "unit_cost_usd_per_mm_median":"custo_unit_mediana_usd_por_mm",
        "vcpu_hours_mean":"vcpu_mean_h","vcpu_hours_sum":"vcpu_total_h",
        "memory_gb_hours_mean":"mem_mean_gb_h","memory_gb_hours_sum":"mem_total_gb_h",
        "shuffle_gb_mean":"shuffle_mean_gb","shuffle_gb_sum":"shuffle_total_gb"
    })
    summary_ds_path = OUT_DIR / "resumo_teste_variante_dataset.csv"
    summary_ds.to_csv(summary_ds_path, index=False)
    summary_ds.head(10)


In [ ]:
def ci_table(df_in: pd.DataFrame, group_cols, metric: str) -> pd.DataFrame:
    rows = []
    for keys, part in df_in.groupby(group_cols, dropna=False):
        a = part[metric].to_numpy(dtype=float)
        mean, lo, hi, n = ci_mean_t(a, alpha=0.05)
        row = dict(zip(group_cols, keys if isinstance(keys, tuple) else (keys,)))
        row.update({"metric": metric, "n": n, "mean": mean, "ci95_lo": lo, "ci95_hi": hi})
        rows.append(row)
    return pd.DataFrame(rows)

ci_metrics = ["duration_s","throughput_rps","total_cost_usd","unit_cost_usd_per_mm"]
ci_frames = []
for m in ci_metrics:
    if m in df.columns:
        ci_frames.append(ci_table(df, ["test","variant"], m))
ci_all = pd.concat(ci_frames, ignore_index=True) if ci_frames else pd.DataFrame()
ci_path = OUT_DIR / "ci95_por_teste_variante.csv"
ci_all.to_csv(ci_path, index=False)
ci_all.head(12)


In [ ]:
def pairwise_tests_per_test(df_in: pd.DataFrame, metric: str) -> pd.DataFrame:
    out = []
    for test_name, g in df_in.groupby("test"):
        variants = list(g["variant"].dropna().unique())
        if len(variants) != 2:
            continue  # esperado 2 variantes por teste
        v1, v2 = variants[0], variants[1]
        x = g.loc[g["variant"]==v1, metric].to_numpy(dtype=float)
        y = g.loc[g["variant"]==v2, metric].to_numpy(dtype=float)
        res = test_two_groups(x, y)
        out.append({
            "test": test_name, "metric": metric,
            "variant_a": v1, "variant_b": v2,
            "nx": res["nx"], "ny": res["ny"],
            "test_used": res["test"], "stat": res["stat"],
            "pvalue": res["pvalue"], "effect": res["effect"], "effect_name": res["effect_name"],
            "mean_a": float(np.nanmean(x)) if x.size>0 else np.nan,
            "mean_b": float(np.nanmean(y)) if y.size>0 else np.nan,
            "delta_abs": (float(np.nanmean(x)) - float(np.nanmean(y))) if x.size>0 and y.size>0 else np.nan,
            "delta_pct": ((float(np.nanmean(x)) / float(np.nanmean(y)) - 1)*100.0)
                         if x.size>0 and y.size>0 and np.nanmean(y)!=0 else np.nan
        })
    return pd.DataFrame(out)

test_metrics = ["duration_s","throughput_rps","total_cost_usd","unit_cost_usd_per_mm"]
tests_all = []
for m in test_metrics:
    if m in df.columns:
        tests_all.append(pairwise_tests_per_test(df, m))
tests_all = pd.concat(tests_all, ignore_index=True) if tests_all else pd.DataFrame()
tests_path = OUT_DIR / "tests_pairwise_por_teste.csv"
tests_all.to_csv(tests_path, index=False)
tests_all


In [ ]:
def plot_pairwise_per_test(df_summary, test_name, metrics):
    sub = df_summary[df_summary["test"]==test_name]
    for col, title, ylabel, fname in metrics:
        if col not in sub.columns: continue
        plt.figure(figsize=(6,4))
        plt.bar(sub["variant"], sub[col].values)
        plt.ylabel(ylabel)
        plt.title(f"{test_name} — {title}")
        plt.tight_layout()
        plt.savefig(OUT_DIR / f"{test_name}_{fname}.png")
        plt.close()

metrics = [
    ("dur_mean_s", "Duração média (s)", "s", "duracao.png"),
    ("thr_mean_rps", "Throughput médio (reg/s)", "reg/s", "throughput.png"),
    ("custo_unit_mean_usd_por_mm", "Custo unitário médio (USD por 1M)", "USD/1M", "custo_unit.png"),
]

for t in summary["test"].unique():
    plot_pairwise_per_test(summary, t, metrics)


In [ ]:
def percent_diffs(df_in: pd.DataFrame, metric: str) -> pd.DataFrame:
    res = []
    for t, g in df_in.groupby("test"):
        part = g[["variant", metric]].dropna()
        if part.empty or part["variant"].nunique()!=2:
            continue
        means = part.groupby("variant")[metric].mean()
        pairs = means.index.tolist()
        if len(pairs)!=2:
            continue
        a, b = pairs[0], pairs[1]
        ma, mb = means[a], means[b]
        res.append({
            "test": t, "metric": metric,
            "variant_a": a, "mean_a": ma,
            "variant_b": b, "mean_b": mb,
            "delta_abs": ma - mb,
            "delta_pct": ((ma/mb - 1)*100.0) if mb!=0 else np.nan
        })
    return pd.DataFrame(res)

pct_frames = []
for m in ["duration_s","throughput_rps","total_cost_usd","unit_cost_usd_per_mm"]:
    if m in df.columns:
        pct_frames.append(percent_diffs(df, m))
pct_all = pd.concat(pct_frames, ignore_index=True) if pct_frames else pd.DataFrame()
pct_path = OUT_DIR / "percent_diffs.csv"
pct_all.to_csv(pct_path, index=False)
pct_all


In [ ]:
# Prepara labels
sv = summary.copy()
sv["label"] = sv["test"] + "—" + sv["variant"]

def bar_metric(sv: pd.DataFrame, col: str, title: str, ylabel: str, fname: str):
    if col not in sv.columns: 
        print(f"[skip] {col} ausente no resumo.")
        return
    plt.figure(figsize=(10,6))
    plt.bar(sv["label"], sv[col].values)
    plt.xticks(rotation=45, ha="right")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.tight_layout()
    plt.savefig(OUT_DIR / fname)
    plt.close()

bar_metric(sv, "custo_unit_mean_usd_por_mm", "Custo unitário médio (USD por 1M)", "USD por 1M",
           "plot_custo_unit_medio.png")
bar_metric(sv, "thr_mean_rps", "Throughput médio (reg/s)", "reg/s",
           "plot_throughput_medio.png")
bar_metric(sv, "dur_mean_s", "Duração média (s)", "s",
           "plot_duracao_media.png")

print("Gráficos salvos em:", OUT_DIR.resolve())


In [ ]:
def flag_outliers(df_in: pd.DataFrame, group_cols, metric: str) -> pd.DataFrame:
    rows = []
    for keys, part in df_in.groupby(group_cols, dropna=False):
        x = part[metric].astype(float)
        if x.dropna().empty:
            continue
        q1, q3 = np.nanpercentile(x, 25), np.nanpercentile(x, 75)
        iqr = q3 - q1
        lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
        mask = (x < lo) | (x > hi)
        cols = [c for c in ["job_name","run"] if c in part.columns]
        tmp = part.loc[mask, group_cols + [metric] + cols].copy()
        if not tmp.empty:
            tmp["lo"], tmp["hi"] = lo, hi
            rows.append(tmp)
    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()

out_dur = flag_outliers(df, ["test","variant"], "duration_s") if "duration_s" in df.columns else pd.DataFrame()
out_thr = flag_outliers(df, ["test","variant"], "throughput_rps") if "throughput_rps" in df.columns else pd.DataFrame()
out_cum = flag_outliers(df, ["test","variant"], "unit_cost_usd_per_mm") if "unit_cost_usd_per_mm" in df.columns else pd.DataFrame()

out_dur.to_csv(OUT_DIR / "possiveis_outliers_duracao.csv", index=False)
out_thr.to_csv(OUT_DIR / "possiveis_outliers_throughput.csv", index=False)
out_cum.to_csv(OUT_DIR / "possiveis_outliers_custo_unit.csv", index=False)

out_dur.head(10)


In [ ]:
eff = df.copy()
# Frações de custo por componente (quando disponíveis)
for col in ["cost_vcpu_usd","cost_memory_usd","cost_shuffle_usd","total_cost_usd"]:
    if col in eff.columns:
        eff[col] = pd.to_numeric(eff[col], errors="coerce")

if {"cost_vcpu_usd","cost_memory_usd","cost_shuffle_usd","total_cost_usd"}.issubset(eff.columns):
    eff["cost_vcpu_frac"]   = eff["cost_vcpu_usd"]   / eff["total_cost_usd"]
    eff["cost_memory_frac"] = eff["cost_memory_usd"] / eff["total_cost_usd"]
    eff["cost_shuffle_frac"]= eff["cost_shuffle_usd"]/ eff["total_cost_usd"]

# $ por vCPU-h e $ por GB-h de memória
if {"total_cost_usd","vcpu_hours"}.issubset(eff.columns):
    eff["usd_per_vcpu_h"] = eff["total_cost_usd"] / eff["vcpu_hours"]
if {"total_cost_usd","memory_gb_hours"}.issubset(eff.columns):
    eff["usd_per_mem_gb_h"] = eff["total_cost_usd"] / eff["memory_gb_hours"]

# $ por GB de shuffle
if {"total_cost_usd","shuffle_gb"}.issubset(eff.columns):
    eff["usd_per_shuffle_gb"] = eff["total_cost_usd"] / eff["shuffle_gb"]

# Agregamento por Teste–Variante
kpis = ["usd_per_vcpu_h","usd_per_mem_gb_h","usd_per_shuffle_gb",
        "cost_vcpu_frac","cost_memory_frac","cost_shuffle_frac"]
present_kpis = [k for k in kpis if k in eff.columns]

if present_kpis:
    eff_summary = eff.groupby(["test","variant"], dropna=False)[present_kpis].mean().reset_index()
    eff_path = OUT_DIR / "eficiencia_recursos_teste_variante.csv"
    eff_summary.to_csv(eff_path, index=False)
    eff_summary
else:
    print("Sem colunas de KPIs")


In [ ]:
print(469.35096158333334 - 468.9694518333333)